# Names 
Spencer Bernardo-Cheng
Max Goldstein
Robbie Keehan

# Data Preparation

In [275]:
import numpy as np
import pandas as pd
datafb = pd.read_csv("./FB.csv")
datagoogle = pd.read_csv("./GOOGL.csv")
datamsft = pd.read_csv("./MSFT.csv")
dataam = pd.read_csv("./AMZN (1).csv")
datanflx = pd.read_csv("./NFLX.csv")

del datafb['Date']
del datagoogle['Date']
del datamsft['Date']
del datanflx['Date']


data = pd.concat([datafb, datagoogle, datamsft,datanflx, dataam ], axis=1, sort=False)
data = data[90:]
del data['Change']
del data['Gain']
del data['Loss']
del data['Average Gain']
del data['Average Loss']
del data['RS']
del data['DM+1']
del data['DM-1']
del data['TR-14']
del data['DM+1-14']
del data['DM-1+14']
del data['dl+1-14']
del data['dl-1-14']
del data['dl14diff']
del data['dl14sum']
del data['DX']
del data['TR']

predictedDays = 14
up = .03
down =-.03
ydata = data['% Change']
ydata = ydata.apply(lambda x:2 if x>=up else (1 if (x< up and x > down) else 0))
ydata = pd.DataFrame(ydata)
ydata['newy'] = ""
# created target data in format to match output of multiple timesteps of 14 days ahead
# [0,1,2,3,4,5,6,7]
# [1,2,3,4,5,6,7,8]
for index, row in ydata.iterrows():
    temp = np.array(ydata['% Change'][index-90:index-90+predictedDays].values)
    ydata['newy'].at[index] = temp
    
# removed last 14 because their targets are not able to created
ydata = ydata[0:-(predictedDays-1)]
data = data[0:-(predictedDays-1)]
del ydata['% Change']
del data['Date']

colheaders = data.columns[0:-3]

for header in colheaders: 
    for i in range(1, len(data)):
        data[header].at[90+i-1]= data[header].iloc[i] - data[header].iloc[i - 1]
        
data = data[0:-1]
ydata= ydata[0:-1]
print(data)

       Open-fb   High-fb    Low-fb  Close-fb  Adj Close-fb  Volume-fb  \
90   -0.459999 -0.609998 -0.519998 -0.590001     -0.590001   12160000   
91   -0.050001  0.019998  0.160000  0.110001      0.110001  -17504500   
92   -0.129999 -0.160000 -0.130001 -0.230000     -0.230000   -3007900   
93   -0.070000  0.080000  0.010000  0.000000      0.000000    1380300   
94    0.000000 -0.189998 -0.190001 -0.040000     -0.040000    1645000   
...        ...       ...       ...       ...           ...        ...   
1873  0.070007  4.979996  1.200012  4.860000      4.860000    8984300   
1874  4.699997  0.639999  3.000000 -1.279999     -1.279999   -6754800   
1875 -1.770004 -1.669998 -1.290008 -0.040008     -0.040008   -1820200   
1876  1.330002  1.270004  1.930008  1.950012      1.950012    2483800   
1877  0.300003  3.330002 -0.330002  2.299988      2.299988    4642900   

      Open-googl  High-googl  Low-googl  Close-googl  ...   Low-amzn  \
90     -8.913910   -6.901916  -2.122101     0.23523

# One Hot Encoding The Target Data

In [270]:
from keras.utils.np_utils import to_categorical
print(ydata.head())
ydata['newy'] = ydata['newy'].apply(lambda x: to_categorical(x,3))
print(ydata)

                                          newy
90  [0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2]
91  [0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0]
92  [0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0]
93  [0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2]
94  [2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2]
                                                   newy
90    [[1.0, 0.0, 0.0], [1.0, 0.0, 0.0], [1.0, 0.0, ...
91    [[1.0, 0.0, 0.0], [1.0, 0.0, 0.0], [1.0, 0.0, ...
92    [[1.0, 0.0, 0.0], [1.0, 0.0, 0.0], [0.0, 0.0, ...
93    [[1.0, 0.0, 0.0], [0.0, 0.0, 1.0], [1.0, 0.0, ...
94    [[0.0, 0.0, 1.0], [1.0, 0.0, 0.0], [0.0, 0.0, ...
...                                                 ...
1873  [[1.0, 0.0, 0.0], [0.0, 0.0, 1.0], [1.0, 0.0, ...
1874  [[0.0, 0.0, 1.0], [1.0, 0.0, 0.0], [0.0, 0.0, ...
1875  [[1.0, 0.0, 0.0], [0.0, 0.0, 1.0], [1.0, 0.0, ...
1876  [[0.0, 0.0, 1.0], [1.0, 0.0, 0.0], [0.0, 0.0, ...
1877  [[1.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, ...

[1788 rows x 1 columns]


# Min-Max Scaling the Time Series

In [271]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(data.iloc[:,:].values)
print(x_scaled)


[[0.6704866  0.63964878 0.66867271 ... 0.31321402 0.35811382 0.15022488]
 [0.67729607 0.65028702 0.68004011 ... 0.34297397 0.26414926 0.12331708]
 [0.67596742 0.64724754 0.67519221 ... 0.42513468 0.25300192 0.10448737]
 ...
 [0.64872934 0.62174942 0.65580059 ... 0.38193301 0.3029723  0.1344608 ]
 [0.7002159  0.67139486 0.709629   ... 0.44102474 0.31038595 0.1153046 ]
 [0.68310912 0.70618038 0.67184883 ... 0.40337188 0.25243424 0.13533161]]


# Utilizing Time Series Generator

In [272]:
from keras.models import Sequential
from keras.layers import Dense
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM, Lambda
from keras.preprocessing.sequence import TimeseriesGenerator

n_input = 2
series = np.array(x_scaled)
target = np.array(ydata)
generator = TimeseriesGenerator(series, target, length=90, batch_size=1)
xtime = []
ytarget =[]
for i in range(len(generator)):
    x, y = generator[i]
    xtime.append(x[0])
    ytarget.append(y[0][0])
xtime = np.array(xtime)
ytarget = np.array(ytarget)



# Time Series Fold Splits

In [273]:
from sklearn.model_selection import train_test_split
import keras.utils
from keras import utils as np_utils
x_train,x_test,y_train,y_test = train_test_split(xtime, ytarget, test_size = 1/4, random_state = 1)

print(x_train.shape)
print(y_train.shape)
print(y_test.shape)

(1273, 90, 36)
(1273, 14, 3)
(425, 14, 3)


In [274]:

model = Sequential()
model.add(LSTM(100, dropout=.2,activation='relu', input_shape=(90, 36), return_sequences=True))
model.add(Lambda(lambda x: x[:, -predictedDays:, :]))
model.add(Dense(3,activation='softmax'))

model.summary()
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])

# fit model
model.fit(x_train, y_train, validation_data=(x_test,y_test), shuffle=True, epochs=200, verbose=1)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_31 (LSTM)               (None, 90, 100)           54800     
_________________________________________________________________
lambda_16 (Lambda)           (None, 14, 100)           0         
_________________________________________________________________
dense_28 (Dense)             (None, 14, 3)             303       
Total params: 55,103
Trainable params: 55,103
Non-trainable params: 0
_________________________________________________________________
Train on 1273 samples, validate on 425 samples
Epoch 1/200
1273/1273 [==============================] - 7s 6ms/step - loss: 1.0486 - acc: 0.5159 - val_loss: 0.8354 - val_acc: 0.5345
Epoch 2/200
1273/1273 [==============================] - 4s 3ms/step - loss: 0.8448 - acc: 0.5190 - val_loss: 0.8169 - val_acc: 0.5345
Epoch 3/200
1273/1273 [==============================] - 4s 3ms/step - loss: 0.7844 -

1273/1273 [==============================] - 4s 3ms/step - loss: 8.1451 - acc: 0.4946 - val_loss: 8.2216 - val_acc: 0.4899
Epoch 56/200
1273/1273 [==============================] - 4s 3ms/step - loss: 8.2012 - acc: 0.4903 - val_loss: 8.0211 - val_acc: 0.5024
Epoch 57/200
1273/1273 [==============================] - 4s 3ms/step - loss: 7.9650 - acc: 0.5058 - val_loss: 8.0320 - val_acc: 0.5017
Epoch 58/200
1273/1273 [==============================] - 4s 3ms/step - loss: 8.1259 - acc: 0.4955 - val_loss: 8.2785 - val_acc: 0.4864
Epoch 59/200
1273/1273 [==============================] - 4s 3ms/step - loss: 7.6021 - acc: 0.5281 - val_loss: 7.5037 - val_acc: 0.5345
Epoch 60/200
1273/1273 [==============================] - 4s 3ms/step - loss: 7.5327 - acc: 0.5327 - val_loss: 7.5037 - val_acc: 0.5345
Epoch 61/200
1273/1273 [==============================] - 4s 3ms/step - loss: 7.5327 - acc: 0.5327 - val_loss: 7.5037 - val_acc: 0.5345
Epoch 62/200
1273/1273 [==============================] - 4s 

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt
#From Sklearn documentation 
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = list(unique_labels(y_true, y_pred))
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
class_names = ['down','neutral','up']
yhat = model.predict(x_test)

ytest2d = y_test.reshape((predictedDays*428,3))
yhat2d = yhat.reshape((predictedDays*428,3))

ytest2d = ytest2d.argmax(axis=1)
yhat2d=yhat2d.argmax(axis=1)
plot_confusion_matrix(ytest2d, yhat2d, classes=class_names,
                      title='SciKit Learn Implementation')

In [ ]:
pred = model.predict(x_test[243][np.newaxis,:,:])
print(x_test[0].shape)
print(x_test.shape)

print(pred)
